In [1]:
%aiida
from aiida.orm.data.upf import get_pseudos_from_structure
import sys

In [8]:
def get_numelectrons_structure_upffamily(structure, pseudos):
    
    def parse_numelectrons_upfpath(upfpath):
        for line in open(upfpath):
            if "valence" in line.lower() and "z" in line.lower():
                if len(line.split("=")) == 2:
                    num_e = int(float((line.split("=")[-1].strip().strip('"'))))
                elif len(line.split()) == 3:
                    num_e = int(float(line.split()[0]))
                else:
                    raise Exception("Could not parse {}".format(upfpath))
        return num_e

    
    def build_upf_numelectrons_dict(structure_ase, pseudos):
        element_nume_dict = {}
        for element in set(structure_ase.get_chemical_symbols()):
            upfpath = pseudos[element].get_file_abs_path()
            element_nume_dict[element] = parse_numelectrons_upfpath(upfpath)
        return element_nume_dict

    
    structure_ase = structure.get_ase()
    
    element_nume_dict = build_upf_numelectrons_dict(structure_ase, pseudos)
    
    num_e = 0
    for element in structure_ase.get_chemical_symbols():
        num_e += element_nume_dict[element]
        
    return num_e    

In [9]:
#we should get 96 electrons
structure = load_node(15586)
upf_family = "SSSPefv1.1_alalloy"
pseudos = get_pseudos_from_structure(structure, upf_family)

%timeit get_numelectrons_structure_upffamily(structure, pseudos)

100 loops, best of 3: 16.3 ms per loop
